In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import oenotebook as oenb
from openeye import oechem, oeiupac
from fragmenter import chemi, torsions
import numpy as np
import seaborn as sbn
from matplotlib.backends.backend_pdf import PdfPages
import cmiles
import json
import socket
import uuid
import getpass

In [2]:
# load all molecules
all_mols = chemi.file_to_oemols('../phenyls.smi')
all_mols.extend(chemi.file_to_oemols('../pyridine_ortho.smi'))
all_mols.extend(chemi.file_to_oemols('../pyridine_meta.smi'))
all_mols.extend(chemi.file_to_oemols('../pyridine_para.smi'))

In [3]:
# Only look at fgroups that have a torsion (skip halogens, phenoxide)
fgroups_to_prep = ['benzoicacid', 'phenylurea', 'dimethylamino', 'methylamino', 'amino', 
                   'ethylamino', 'propylamino', 'hydroxy', 'methoxy', 'ethoxy', 'methyl', 'ethoxycarbonyl', 
                   'nitro', 'amide', 'ethylamide', 'urea', 'dimethylurea','carbamate']

In [4]:
def group_by_fgroup_and_wbo(fgroup, molecules):
    mols = []
    for mol in molecules:
        name = mol.GetTitle().split('_')
        if fgroup in name:
            mols.append(mol)
            
    # remove duplicates
    smiles = []
    deduplicated_mols = []
    for mol in mols:
        sm = oechem.OEMolToSmiles(mol)
        if sm not in smiles:
            smiles.append(sm)
            deduplicated_mols.append(mol)
    # sort by wbo
    wbo_dict = {}
    for mol in deduplicated_mols:
        name = mol.GetTitle().split('_')
        drop = False
        if name[-1] == fgroup:
            to_check_ortho = [1, 4, 5]
        elif name[1] == fgroup:
            to_check_ortho = [2, 3, 4, 5]
        for bond in mol.GetBonds():
            a1 = bond.GetBgn()
            a2 = bond.GetEnd()
            m1 = a1.GetMapIdx()
            m2 = a2.GetMapIdx()
            if (m1 in to_check_ortho or m2 in to_check_ortho) and bond.IsInRing():
                if a1.GetAtomicNum() == 7 or a2.GetAtomicNum() == 7:
                    drop = True
        if not drop:
            charged = chemi.get_charges(mol)
            if fgroup == name[-1]:
                map_idx = [1]
            elif fgroup == name[1]:
                map_idx = [2, 3]
            for bond in charged.GetBonds():
                m1 = bond.GetBgn().GetMapIdx()
                m2 = bond.GetEnd().GetMapIdx()
                if (m1 in map_idx or m2 in map_idx) and not bond.IsInRing():
                    wbo = bond.GetData('WibergBondOrder')
                    if wbo not in wbo_dict:
                        wbo_dict[wbo] = []
                    wbo_dict[wbo].append(charged)
    return wbo_dict

def select_mols_to_run(wbo_dict, interval=15):
    # Choose molecules to run to get the full range of WBO of central bond
    # Sort keys
    wbo_sorted_keys = sorted(list(wbo_dict.keys()))
    selected = set()
    for wbo in wbo_dict:
        mol = wbo_dict[wbo][0]
        name = mol.GetTitle()
        name = name.split('_')
        if len(name) == 1:
            selected.add(wbo)
    selected.add(wbo_sorted_keys[0])
    selected.add(wbo_sorted_keys[-1])
    selected.update(wbo_sorted_keys[::interval])
    selected = sorted(list(selected))
    mols = [wbo_dict[wbo][0] for wbo in selected]
    return mols

def add_atom_map(mol):
    """
    Will also add explicit H if missing
    Save existing atom map in different atom data because omega conformer generation will overwrite it
    """
    # copy molecule
    mol = oechem.OEMol(mol)
    if not cmiles.utils.has_explicit_hydrogen(mol):
        cmiles.utils.add_explicit_hydrogen(mol)
    cmiles._cmiles_oe.canonical_order_atoms(mol, in_place=True)
    for a in mol.GetAtoms():
        if a.GetMapIdx() != 0 :
            a.SetData('map_idx', a.GetMapIdx())
    cmiles.utils.remove_atom_map(mol, keep_map_data=True)
    # Add map indices on mol
    for a in mol.GetAtoms():
        a.SetMapIdx(a.GetIdx() + 1)
    return mol
    
def find_torsion(mol, fgroup):
    # Find torsion to drive
    name = mol.GetTitle().split('_')
    if fgroup == name[-1]:
        map_idx = [1]
    elif fgroup == name[1]:
        map_idx = [2, 3]
    if cmiles.utils.is_missing_atom_map(mol):
        raise('mol is missing atom map indices')
    tors = torsions.find_torsions(mol, restricted=False, terminal=True)
    for bond in mol.GetBonds():
        m = 0
        a1 = bond.GetBgn()
        a2 = bond.GetEnd()
        if 'map_idx' in a1.GetData():
            m = a1.GetData('map_idx')
        if 'map_idx' in a2.GetData():
            m = a2.GetData('map_idx')
        if m in map_idx and not bond.IsInRing():
            # Find all the torsions around this bond
            central_bond = (a1.GetMapIdx() - 1, a2.GetMapIdx()-1)
    for tor_type in tors:
        for tor in tors[tor_type]:
            torsion = tors[tor_type][tor]
            if central_bond == (torsion[1], torsion[2]) or central_bond == (torsion[2], torsion[1]):
                tor_to_drive = torsion
    return tor_to_drive

def generate_torsiondrive_input(mol, tor_to_drive, grid_spacing=15):
    #copy mol
    mol_copy = oechem.OEMol(mol)
    # Add explicit hydrogen and atom map
    #mol_copy = add_atom_map(mol)
    # To generate mol ids, first generate an explicit hydrogen SMILES
    smiles = cmiles.utils.mol_to_smiles(mol_copy, mapped=False)
    # Generate mol id
    mol_id = cmiles.to_molecule_id(smiles)
    # find the torsion of interest to drive
    #tor_to_drive = find_torsion(mol_copy, fgroup='benzoicacid')
    # Generate intitial conformers. I'm not doing a grid because these torsions are not congested and don't 
    # have intramolecular interactions as far as I can tell. 
    conformers = chemi.generate_conformers(mol_copy, dense=True)
    qcschema_molecules = [cmiles.utils.mol_to_map_ordered_qcschema(conf, mol_id) for conf in conformers.GetConfs()]
    torsiondrive_job_name = str(tor_to_drive[0]) + '_' + str(tor_to_drive[1]) + '_' + str(tor_to_drive[2]) + '_' + str(tor_to_drive[3])
    torsiondrive_input = {mol_id['canonical_isomeric_explicit_hydrogen_mapped_smiles']:{
                          torsiondrive_job_name: {'type': 'torsiondrive_input',
                                                  'initial_molecule': qcschema_molecules,
                                                  'dihedrals': [tor_to_drive],
                                                  'grid_spacing': [grid_spacing]},

                           'provenance': {}}}
    return torsiondrive_input

In [8]:
for fgroup in fgroups_to_prep:
    wbo_dict = group_by_fgroup_and_wbo(fgroup=fgroup, molecules=all_mols)
    mols_to_run = select_mols_to_run(wbo_dict)
    torsiondrive_inputs = {}
    for mol in mols_to_run:
        # Add atom map
        mol_copy = add_atom_map(mol)
        # Find torsion of interest
        tor_to_drive = find_torsion(mol_copy, fgroup=fgroup)
        td_input = generate_torsiondrive_input(mol_copy, tor_to_drive)
        torsiondrive_inputs[list(td_input.keys())[0]] = td_input[list(td_input.keys())[0]]
    with open('{}_torsiondrive_inputs.json'.format(fgroup), 'w') as f:
        json.dump(torsiondrive_inputs, f, indent=2, sort_keys=True)



21:24:05 INFO fragmenter: List of torsion to drive:
21:24:05 INFO fragmenter: Idx: 9 11 7 17
21:24:05 INFO fragmenter: Atom numbers: 6 7 6 1
21:24:05 INFO fragmenter: Idx: 9 11 8 22
21:24:05 INFO fragmenter: Atom numbers: 6 7 6 1
21:24:05 INFO fragmenter: Idx: 8 11 9 25
21:24:05 INFO fragmenter: Atom numbers: 6 7 6 1
21:24:05 INFO fragmenter: Idx: 12 6 13 26
21:24:05 INFO fragmenter: Atom numbers: 8 6 8 1
21:24:05 INFO fragmenter: List of torsion to drive:
21:24:05 INFO fragmenter: Idx: 1 4 6 12
21:24:05 INFO fragmenter: Atom numbers: 6 6 6 8
21:24:05 INFO fragmenter: Idx: 10 5 11 7
21:24:05 INFO fragmenter: Atom numbers: 7 6 7 6
21:24:05 INFO fragmenter: List of torsion to drive:
21:24:05 INFO fragmenter: Idx: 5 9 7 15
21:24:05 INFO fragmenter: Atom numbers: 6 7 6 1


21:24:05 INFO fragmenter: Idx: 10 6 11 19
21:24:05 INFO fragmenter: Atom numbers: 8 6 8 1
21:24:05 INFO fragmenter: List of torsion to drive:
21:24:05 INFO fragmenter: Idx: 1 4 6 10
21:24:05 INFO fragmenter: Atom numbers: 6 6 6 8
21:24:05 INFO fragmenter: Idx: 8 5 9 7
21:24:05 INFO fragmenter: Atom numbers: 7 6 7 6
21:24:06 INFO fragmenter: List of torsion to drive:
21:24:06 INFO fragmenter: Idx: 8 6 9 14
21:24:06 INFO fragmenter: Atom numbers: 8 6 8 1
21:24:06 INFO fragmenter: List of torsion to drive:
21:24:06 INFO fragmenter: Idx: 1 5 6 8
21:24:06 INFO fragmenter: Atom numbers: 6 6 6 8
21:24:06 INFO fragmenter: List of torsion to drive:
21:24:06 INFO fragmenter: Idx: 8 7 9 15
21:24:06 INFO fragmenter: Atom numbers: 8 6 8 1
21:24:06 INFO fragmenter: List of torsion to drive:
21:24:06 INFO fragmenter: Idx: 1 5 7 8
21:24:06 INFO fragmenter: Atom numbers: 6 6 6 8
21:24:06 INFO fragmenter: List of torsion to drive:
21:24:06 INFO fragmenter: Idx: 8 12 9 20
21:24:06 INFO fragmenter: Atom n

21:25:09 INFO fragmenter: List of torsion to drive:
21:25:09 INFO fragmenter: Idx: 11 7 8 16
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 1
21:25:09 INFO fragmenter: List of torsion to drive:
21:25:09 INFO fragmenter: Idx: 1 5 9 7
21:25:09 INFO fragmenter: Atom numbers: 6 6 7 6
21:25:09 INFO fragmenter: Idx: 11 7 9 5
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 6
21:25:09 INFO fragmenter: List of torsion to drive:
21:25:09 INFO fragmenter: Idx: 7 5 8 15
21:25:09 INFO fragmenter: Atom numbers: 7 6 7 1
21:25:09 INFO fragmenter: Idx: 11 6 9 17
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 1
21:25:09 INFO fragmenter: List of torsion to drive:
21:25:09 INFO fragmenter: Idx: 1 4 10 6
21:25:09 INFO fragmenter: Atom numbers: 6 6 7 6
21:25:09 INFO fragmenter: Idx: 11 6 10 4
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 6
21:25:09 INFO fragmenter: List of torsion to drive:
21:25:09 INFO fragmenter: Idx: 10 7 8 16
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 1
21:25:09 INFO fragmenter: List of

21:25:09 INFO fragmenter: Idx: 1 5 9 7
21:25:09 INFO fragmenter: Atom numbers: 6 6 7 6
21:25:09 INFO fragmenter: Idx: 10 7 9 5
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 6
21:25:09 INFO fragmenter: List of torsion to drive:
21:25:09 INFO fragmenter: Idx: 15 10 9 20
21:25:09 INFO fragmenter: Atom numbers: 8 6 6 1
21:25:09 INFO fragmenter: Idx: 14 8 11 25
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 1
21:25:09 INFO fragmenter: List of torsion to drive:
21:25:09 INFO fragmenter: Idx: 1 5 7 13
21:25:09 INFO fragmenter: Atom numbers: 6 6 6 8
21:25:09 INFO fragmenter: Idx: 3 6 12 8
21:25:09 INFO fragmenter: Atom numbers: 6 6 7 6
21:25:09 INFO fragmenter: Idx: 14 8 12 6
21:25:09 INFO fragmenter: Atom numbers: 8 6 7 6
21:25:09 INFO fragmenter: Idx: 23 10 15 7
21:25:09 INFO fragmenter: Atom numbers: 1 6 8 6
21:25:09 INFO fragmenter: Idx: 5 7 15 10
21:25:09 INFO fragmenter: Atom numbers: 6 6 8 6
21:25:09 INFO fragmenter: Idx: 24 10 15 7
21:25:09 INFO fragmenter: Atom numbers: 1 6 8 6
21:25:0

21:26:00 INFO fragmenter: List of torsion to drive:
21:26:00 INFO fragmenter: Idx: 5 11 7 17
21:26:00 INFO fragmenter: Atom numbers: 6 7 6 1
21:26:00 INFO fragmenter: Idx: 5 11 8 20
21:26:00 INFO fragmenter: Atom numbers: 6 7 6 1
21:26:00 INFO fragmenter: Idx: 10 12 9 23
21:26:00 INFO fragmenter: Atom numbers: 6 7 6 1
21:26:00 INFO fragmenter: Idx: 6 12 10 27
21:26:00 INFO fragmenter: Atom numbers: 6 7 6 1
21:26:00 INFO fragmenter: List of torsion to drive:
21:26:00 INFO fragmenter: Idx: 2 5 11 7
21:26:00 INFO fragmenter: Atom numbers: 6 6 7 6
21:26:00 INFO fragmenter: Idx: 3 6 12 9
21:26:00 INFO fragmenter: Atom numbers: 6 6 7 6
21:26:00 INFO fragmenter: List of torsion to drive:
21:26:00 INFO fragmenter: Idx: 5 9 7 15
21:26:00 INFO fragmenter: Atom numbers: 6 7 6 1
21:26:00 INFO fragmenter: Idx: 5 9 8 18
21:26:00 INFO fragmenter: Atom numbers: 6 7 6 1
21:26:00 INFO fragmenter: List of torsion to drive:
21:26:00 INFO fragmenter: Idx: 2 5 9 7
21:26:00 INFO fragmenter: Atom numbers: 6 6

...

21:39:15 INFO fragmenter: List of torsion to drive:
21:39:15 INFO fragmenter: Idx: 7 10 8 17
21:39:15 INFO fragmenter: Atom numbers: 6 7 6 1
21:39:15 INFO fragmenter: Idx: 3 6 12 22
21:39:15 INFO fragmenter: Atom numbers: 6 6 8 1
21:39:15 INFO fragmenter: List of torsion to drive:
21:39:15 INFO fragmenter: Idx: 1 5 9 7
21:39:15 INFO fragmenter: Atom numbers: 6 6 7 6
21:39:15 INFO fragmenter: Idx: 11 7 9 5
21:39:15 INFO fragmenter: Atom numbers: 8 6 7 6
21:39:15 INFO fragmenter: Idx: 11 7 10 8
21:39:15 INFO fragmenter: Atom numbers: 8 6 7 6
21:39:15 INFO fragmenter: List of torsion to drive:
21:39:15 INFO fragmenter: Idx: 7 11 9 21
21:39:15 INFO fragmenter: Atom numbers: 6 6 6 1
21:39:15 INFO fragmenter: Idx: 8 14 10 24
21:39:15 INFO fragmenter: Atom numbers: 6 7 6 1
21:39:15 INFO fragmenter: List of torsion to drive:
21:39:15 INFO fragmenter: Idx: 15 7 11 9
21:39:15 INFO fragmenter: Atom numbers: 8 6 6 6
21:39:15 INFO fragmenter: Idx: 15 7 12 5
21:39:15 INFO fragmenter: Atom numbers: 8

21:41:55 INFO fragmenter: List of torsion to drive:
21:41:55 INFO fragmenter: Idx: 7 11 8 18
21:41:55 INFO fragmenter: Atom numbers: 6 7 6 1
21:41:55 INFO fragmenter: Idx: 7 11 9 21
21:41:55 INFO fragmenter: Atom numbers: 6 7 6 1
21:41:55 INFO fragmenter: List of torsion to drive:
21:41:55 INFO fragmenter: Idx: 1 5 10 7
21:41:55 INFO fragmenter: Atom numbers: 6 6 7 6
21:41:55 INFO fragmenter: Idx: 13 7 10 5
21:41:55 INFO fragmenter: Atom numbers: 8 6 7 6
21:41:55 INFO fragmenter: Idx: 13 7 11 8
21:41:55 INFO fragmenter: Atom numbers: 8 6 7 6
21:41:56 INFO fragmenter: List of torsion to drive:
21:41:56 INFO fragmenter: Idx: 13 11 8 22
21:41:56 INFO fragmenter: Atom numbers: 7 6 6 1
21:41:56 INFO fragmenter: Idx: 10 14 9 25
21:41:56 INFO fragmenter: Atom numbers: 6 7 6 1
21:41:56 INFO fragmenter: Idx: 9 14 10 28
21:41:56 INFO fragmenter: Atom numbers: 6 7 6 1
21:41:56 INFO fragmenter: List of torsion to drive:
21:41:56 INFO fragmenter: Idx: 2 5 12 31
21:41:56 INFO fragmenter: Atom number

21:41:56 INFO fragmenter: Atom numbers: 6 6 7 6
21:41:56 INFO fragmenter: Idx: 8 11 13 6
21:41:56 INFO fragmenter: Atom numbers: 6 6 7 6
21:41:56 INFO fragmenter: Idx: 4 6 13 11
21:41:56 INFO fragmenter: Atom numbers: 6 6 7 6
21:41:56 INFO fragmenter: Idx: 15 7 14 9
21:41:56 INFO fragmenter: Atom numbers: 8 6 7 6
21:41:57 INFO fragmenter: List of torsion to drive:
21:41:57 INFO fragmenter: Idx: 7 11 8 18
21:41:57 INFO fragmenter: Atom numbers: 6 7 6 1
21:41:57 INFO fragmenter: Idx: 7 11 9 21
21:41:57 INFO fragmenter: Atom numbers: 6 7 6 1
21:41:57 INFO fragmenter: List of torsion to drive:
21:41:57 INFO fragmenter: Idx: 4 6 10 7
21:41:57 INFO fragmenter: Atom numbers: 6 6 7 6
21:41:57 INFO fragmenter: Idx: 12 7 10 6
21:41:57 INFO fragmenter: Atom numbers: 8 6 7 6
21:41:57 INFO fragmenter: Idx: 12 7 11 8
21:41:57 INFO fragmenter: Atom numbers: 8 6 7 6
21:41:57 INFO fragmenter: List of torsion to drive:
21:41:57 INFO fragmenter: Idx: 7 12 8 18
21:41:57 INFO fragmenter: Atom numbers: 6 7 

21:43:25 INFO fragmenter: List of torsion to drive:
21:43:25 INFO fragmenter: Idx: 10 7 8 16
21:43:25 INFO fragmenter: Atom numbers: 8 6 7 1
21:43:25 INFO fragmenter: List of torsion to drive:
21:43:25 INFO fragmenter: Idx: 3 6 11 7
21:43:25 INFO fragmenter: Atom numbers: 6 6 8 6
21:43:25 INFO fragmenter: Idx: 10 7 11 6
21:43:25 INFO fragmenter: Atom numbers: 8 6 8 6
21:43:25 INFO fragmenter: List of torsion to drive:
21:43:25 INFO fragmenter: Idx: 9 7 8 16
21:43:25 INFO fragmenter: Atom numbers: 8 6 7 1
21:43:25 INFO fragmenter: List of torsion to drive:
21:43:25 INFO fragmenter: Idx: 4 6 10 7


21:43:25 INFO fragmenter: Atom numbers: 6 6 8 6
21:43:25 INFO fragmenter: Idx: 9 7 10 6
21:43:25 INFO fragmenter: Atom numbers: 8 6 8 6
21:43:26 INFO fragmenter: List of torsion to drive:
21:43:26 INFO fragmenter: Idx: 5 10 8 17
21:43:26 INFO fragmenter: Atom numbers: 6 7 6 1
21:43:26 INFO fragmenter: Idx: 11 7 9 20
21:43:26 INFO fragmenter: Atom numbers: 8 6 7 1
21:43:26 INFO fragmenter: List of torsion to drive:
21:43:26 INFO fragmenter: Idx: 1 5 10 8
21:43:26 INFO fragmenter: Atom numbers: 6 6 7 6
21:43:26 INFO fragmenter: Idx: 3 6 12 7
21:43:26 INFO fragmenter: Atom numbers: 6 6 8 6
21:43:26 INFO fragmenter: Idx: 11 7 12 6
21:43:26 INFO fragmenter: Atom numbers: 8 6 8 6
21:43:26 INFO fragmenter: List of torsion to drive:
21:43:26 INFO fragmenter: Idx: 5 10 7 16
21:43:26 INFO fragmenter: Atom numbers: 6 7 6 1
21:43:26 INFO fragmenter: Idx: 11 6 9 19
21:43:26 INFO fragmenter: Atom numbers: 8 6 7 1
21:43:26 INFO fragmenter: List of torsion to drive:
21:43:26 INFO fragmenter: Idx: 8 5 

In [45]:
username = getpass.getuser()
hostname = socket.gethostname()
collection = 'phenyl_set'
all_inputs = {}
for fgroup in fgroups_to_prep:
    with open('{}_torsiondrive_inputs.json'.format(fgroup), 'r') as f:
        td_inputs = json.load(f)
        for mol in td_inputs:
            if mol not in all_inputs:
                all_inputs[mol] = {}
            provenance = {'username': username,
                          'hostname': hostname,
                          'dataset_name': (collection, fgroup)
                         }
            
            for key in td_inputs[mol]:
                if key != 'provenance':
                    if key not in all_inputs[mol]:
                        all_inputs[mol][key] = {}
                    all_inputs[mol][key] = td_inputs[mol][key]
            all_inputs[mol]['provenance'] = provenance
            

In [46]:
# Write out all molecules
with open('phenyl_set_torsiondrive_input.json', 'w') as f:
    json.dump(all_inputs, f, indent=2, sort_keys=True)

In [47]:
optimization_spec = {
    "program": "geometric",
    "keywords": {
        "coordsys": "tric",
    }
}
qc_spec =  {
    "driver": "gradient",
    "method": "UFF",
    "basis": None,
    "keywords": None,
    "program": "rdkit",
}
rdkit_inputs = {}
for mol in all_inputs:
    if mol not in rdkit_inputs:
        rdkit_inputs[mol] = {}
    for job in all_inputs[mol]:
        if job != 'provenance':
            if job not in rdkit_inputs[mol]:
                rdkit_inputs[mol][job] = {}
            rdkit_inputs[mol][job]['initial_molecule'] = all_inputs[mol][job]['initial_molecule']
            rdkit_inputs[mol][job]['keywords'] = {'dihedrals': all_inputs[mol][job]['dihedrals'],
                                                   'grid_spacing': all_inputs[mol][job]['grid_spacing']}
            rdkit_inputs[mol][job]['optimization_spec'] = optimization_spec
            rdkit_inputs[mol][job]['qc_spec'] = qc_spec
    rdkit_inputs[mol]['provenance'] = all_inputs[mol]['provenance']
                
            

In [49]:
with open('phenyl_set_rdkit_torsiondrive_inputs.json', 'w') as f:
    json.dump(rdkit_inputs, f, indent=2, sort_keys=True)

In [51]:
qc_spec =  {
    "driver": "gradient",
    "method": "B3LYP-D3",
    "basis": "def2-svp",
    "keywords": None,
    "program": "psi4",
}

In [54]:
psi4_inputs = {}
for mol in rdkit_inputs:
    if mol not in psi4_inputs:
        psi4_inputs[mol] = {}
    psi4_inputs[mol]['provenance'] = rdkit_inputs[mol]['provenance']
    for job in rdkit_inputs[mol]:
        if job != 'provenance':
            psi4_inputs[mol][job] = rdkit_inputs[mol][job]
            psi4_inputs[mol][job]['qc_spec'] = qc_spec

In [56]:
with open('phenyl_set_psi4_torsiondrive_inputs.json', 'w') as f:
    json.dump(psi4_inputs, f, indent=2, sort_keys=True)